In [25]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

In [26]:
import os 
os.getcwd()
os.listdir()

'/Users/lizhongtao/myworkspace/SQL优化/设备风险画像监控/设备风险画像-BD维度聚合的新开客户'

['tmp-Behavior_DeviceReset.xlsx',
 '.DS_Store',
 '安装设备异常BD名下聚集客户.xlsx',
 'text_similar.py',
 'tmp2.xlsx',
 'tmp——Behavior_AbnAppInstall.xlsx',
 'tmp-Execution_MultiSys.xlsx',
 'tmp.xlsx',
 'tmp200.xlsx',
 'tmp0926.xlsx',
 'BD维度存在聚集的新开客户_116210770_20250925.xlsx',
 '关联客户超过3个&距离50m以内.xlsx',
 'BD维度聚合的新开客户_优化版.sql',
 'BD维度聚合的新开客户.sql',
 '828首单客户核查结论.xlsx',
 'SQL优化说明.md',
 'BD维度聚合新开客户挖掘.ipynb',
 'distance_calculate.py',
 'tmp0926_2.xlsx']

#### 加载数据

In [27]:
df = pd.read_excel("关联客户超过3个&距离50m以内.xlsx")
df = df[df['new_begin_period'].isin(['近28天新开','近29~56天新开'])]
df.head()

,customer_id,new_begin_period,is_achieve_new_period,first_arranged_ord_dt,first_arranged_ord_date,first_arranged_ord_belong_bd_id,first_bd_name,area_name,org_name,level2Tags,...,is_non_visit_app_after_1arranged,is_loss_newa_cust_cat2_cluster,poi_name,longitude,latitude,delivery_address,poi_longitude,poi_latitude,poi_address,poi_delivery_distance
0,10015657,近28天新开,其他,20250908,2025-09-08,32165337,赵龙贤,上海一区,上海1区2组,"[""Behavior_DeviceReset"",""Behavior_AbnAppInstall""]",...,0.0,0.0,阿里王椒盐牛蛙（泰山店）,121.445374,31.258581,上海盛怡农副产品市场(小外3号)骊山路11号,121.445374,31.258581,上海盛怡农副产品市场(小外3号),0.000000
1,10027152,近29~56天新开,其他,20250811,2025-08-11,30206917,刘宏,中B上海一区,中B上海2组,"[""Behavior_AbnBattery""]",...,0.0,0.0,重庆鸡公煲(鹤望路店),121.325440,31.245862,鹤望路361号大宅超市和饺子馆中间,121.325511,31.245828,鹤望路361号（大宅超市旁）,7.745038
2,10029618,近29~56天新开,其他,20250818,2025-08-18,30265585,傅鑫,中B上海V客销售组,中B上海V客连锁2组,"[""ROM_3rd"",""Behavior_AbnAppInstall"",""Device_Em...",...,0.0,0.0,哈灵现炒本帮面（天山西路店）,121.363920,31.218118,天山西路416号天山西路416号。,121.363791,31.218191,天山西路416号,14.726050
4,10072035,近28天新开,其他,20250914,2025-09-14,30079821,王毫伟,电销三区,NaN,"[""Behavior_DevMode""]",...,0.0,0.0,沙县小吃(新亭西路店),118.842880,31.942877,新亭西路158号（沙县小吃）,118.842862,31.942918,东山街道新亭西路158号小秦淮旁(近秦淮食府),4.870538
6,10118588,近28天新开,其他,20250829,2025-08-29,32358034,唐瑞松,苏州二区,苏州2区2组,"[""Behavior_AbnBattery""]",...,0.0,0.0,阿姣小吃（富利广场店）,120.587356,31.211695,苏州市吴中区富利广场1-27,120.587453,31.211453,吴山街899号富利广场F1层,28.478300


#### 黄线、红线客户占比

In [28]:
# 命中标签的客户中，命中黄线客户占比、命中红线客户占比
print('命中标签的客户中，命中黄线客户占比、命中红线客户占比')
df['is_yellow_line_cust'].sum() / df.loc[df['is_yellow_line_cust'].isin([0,1]),'customer_id'].nunique() # 37.31%命中黄线
df['is_strategy_triggered'].sum() / df.loc[df['is_yellow_line_cust'].isin([0,1]),'customer_id'].nunique() # 18.10%命中红线

命中标签的客户中，命中黄线客户占比、命中红线客户占比


np.float64(0.3759285461620092)

np.float64(0.18199504775380262)

##### BD维度聚合数≥5的BD

In [29]:
tmp = df.groupby(['area_name','org_name','first_bd_name']).agg({'customer_id':"nunique"}).reset_index().sort_values(by='customer_id',ascending=False)
tmp.rename(columns={'customer_id':'cust_cnt'},inplace=True)
cnt5_bd_list = tmp.loc[tmp['cust_cnt'] >= 5,'first_bd_name'].values.tolist()
cnt5_bd_list[:2]

['谢倩', '吕科']

In [30]:
tmp_df = df[df['first_bd_name'].isin(cnt5_bd_list)]
tmp_df = pd.merge(tmp_df, tmp, on=['area_name','org_name','first_bd_name'], how='left')
tmp_df['is_yellow_line_cust'].sum() / tmp_df.loc[tmp_df['is_yellow_line_cust'].isin([0,1]),'customer_id'].nunique() # 41.6%命中黄线
tmp_df['is_strategy_triggered'].sum() / tmp_df.loc[tmp_df['is_yellow_line_cust'].isin([0,1]),'customer_id'].nunique() # 22.7%命中红线

np.float64(0.41402427976082623)

np.float64(0.22685269070483782)

In [31]:
tmp_df['new_begin_period'].value_counts()
tmp_df.loc[tmp_df['new_begin_period']=="近28天新开",'is_achieve_new_period'].value_counts()

new_begin_period
近29~56天新开    2988
近28天新开       2555
Name: count, dtype: int64

is_achieve_new_period
其他       2405
走完新客期     150
Name: count, dtype: int64

In [32]:
def yellow_red_rate(tmp,cnt):
    cnt_bd_list = tmp.loc[tmp['cust_cnt'] >= cnt,'first_bd_name'].values.tolist()
    tmp_df = df[df['first_bd_name'].isin(cnt_bd_list)]
    tmp_df = pd.merge(tmp_df, tmp, on=['area_name','org_name','first_bd_name'], how='left')
    # 仅29~56天新开客户
    tmp_df = tmp_df.loc[tmp_df['is_yellow_line_cust'].isin([0,1])]
    yellow_rate_56d = tmp_df.loc[tmp_df['new_begin_period'] == "近29~56天新开",'is_yellow_line_cust'].sum() / tmp_df.loc[tmp_df['new_begin_period'] == "近29~56天新开",'customer_id'].nunique() 
    red_rate_56d = tmp_df.loc[tmp_df['new_begin_period'] == "近29~56天新开",'is_strategy_triggered'].sum() / tmp_df.loc[tmp_df['new_begin_period'] == "近29~56天新开",'customer_id'].nunique() 
    bd_cust_cnt_56d = tmp_df.loc[tmp_df['new_begin_period'] == "近29~56天新开",'customer_id'].nunique()
    # 仅28天新开客户
    yellow_rate_28d = tmp_df.loc[tmp_df['new_begin_period'] == "近28天新开",'is_yellow_line_cust'].sum() / tmp_df.loc[tmp_df['new_begin_period'] == "近28天新开",'customer_id'].nunique() 
    red_rate_28d = tmp_df.loc[tmp_df['new_begin_period'] == "近28天新开",'is_strategy_triggered'].sum() / tmp_df.loc[tmp_df['new_begin_period'] == "近28天新开",'customer_id'].nunique()
    bd_cust_cnt_28d = tmp_df.loc[tmp_df['new_begin_period'] == "近28天新开",'customer_id'].nunique()

    return len(cnt_bd_list),yellow_rate_56d,red_rate_56d,yellow_rate_28d,red_rate_28d,bd_cust_cnt_56d,bd_cust_cnt_28d
for cnt in range(1,30):
    bd_cnt,yellow_rate_56d,red_rate_56d,yellow_rate_28d,red_rate_28d,bd_cust_cnt_56d,bd_cust_cnt_28d = yellow_red_rate(tmp,cnt)
    print(f"当前阈值 {cnt} 时, 涉及到 {bd_cnt} 个BD, 其中29~56天前新开客户黄线命中率: {yellow_rate_56d:.2%} , 29~56天前新开红线命中率: {red_rate_56d:.2%} ，近28天新开客户黄线命中率: {yellow_rate_28d:.2%} , 近28天新开红线命中率: {red_rate_28d:.2%} ")

当前阈值 1 时, 涉及到 3557 个BD, 其中29~56天前新开客户黄线命中率: 43.62% , 29~56天前新开红线命中率: 20.86% ，近28天新开客户黄线命中率: 30.22% , 近28天新开红线命中率: 14.92% 
当前阈值 2 时, 涉及到 2286 个BD, 其中29~56天前新开客户黄线命中率: 44.82% , 29~56天前新开红线命中率: 21.65% ，近28天新开客户黄线命中率: 30.95% , 近28天新开红线命中率: 15.34% 
当前阈值 3 时, 涉及到 1386 个BD, 其中29~56天前新开客户黄线命中率: 46.13% , 29~56天前新开红线命中率: 23.13% ，近28天新开客户黄线命中率: 31.28% , 近28天新开红线命中率: 16.19% 
当前阈值 4 时, 涉及到 871 个BD, 其中29~56天前新开客户黄线命中率: 47.72% , 29~56天前新开红线命中率: 24.81% ，近28天新开客户黄线命中率: 31.47% , 近28天新开红线命中率: 16.75% 
当前阈值 5 时, 涉及到 573 个BD, 其中29~56天前新开客户黄线命中率: 49.95% , 29~56天前新开红线命中率: 27.21% ，近28天新开客户黄线命中率: 31.36% , 近28天新开红线命中率: 17.37% 
当前阈值 6 时, 涉及到 407 个BD, 其中29~56天前新开客户黄线命中率: 51.25% , 29~56天前新开红线命中率: 28.49% ，近28天新开客户黄线命中率: 31.90% , 近28天新开红线命中率: 18.21% 
当前阈值 7 时, 涉及到 322 个BD, 其中29~56天前新开客户黄线命中率: 52.76% , 29~56天前新开红线命中率: 29.85% ，近28天新开客户黄线命中率: 32.06% , 近28天新开红线命中率: 19.37% 
当前阈值 8 时, 涉及到 270 个BD, 其中29~56天前新开客户黄线命中率: 53.62% , 29~56天前新开红线命中率: 30.43% ，近28天新开客户黄线命中率: 32.06% , 近28天新开红线命中率: 19.47% 
当前阈值 9 时, 涉及到 233 个BD, 其中29~5

#### 按照二级标签展开，统计每个二级标签下新客数， 依次排查客户问题

In [33]:
# 按照二级标签展开
df['level2Tags'] = df['level2Tags'].str.strip('[]').str.replace('"',"").str.split(',')
expand_df= df.explode('level2Tags')
expand_df

,customer_id,new_begin_period,is_achieve_new_period,first_arranged_ord_dt,first_arranged_ord_date,first_arranged_ord_belong_bd_id,first_bd_name,area_name,org_name,level2Tags,...,is_non_visit_app_after_1arranged,is_loss_newa_cust_cat2_cluster,poi_name,longitude,latitude,delivery_address,poi_longitude,poi_latitude,poi_address,poi_delivery_distance
0,10015657,近28天新开,其他,20250908,2025-09-08,32165337,赵龙贤,上海一区,上海1区2组,Behavior_DeviceReset,...,0.0,0.0,阿里王椒盐牛蛙（泰山店）,121.445374,31.258581,上海盛怡农副产品市场(小外3号)骊山路11号,121.445374,31.258581,上海盛怡农副产品市场(小外3号),0.000000
0,10015657,近28天新开,其他,20250908,2025-09-08,32165337,赵龙贤,上海一区,上海1区2组,Behavior_AbnAppInstall,...,0.0,0.0,阿里王椒盐牛蛙（泰山店）,121.445374,31.258581,上海盛怡农副产品市场(小外3号)骊山路11号,121.445374,31.258581,上海盛怡农副产品市场(小外3号),0.000000
1,10027152,近29~56天新开,其他,20250811,2025-08-11,30206917,刘宏,中B上海一区,中B上海2组,Behavior_AbnBattery,...,0.0,0.0,重庆鸡公煲(鹤望路店),121.325440,31.245862,鹤望路361号大宅超市和饺子馆中间,121.325511,31.245828,鹤望路361号（大宅超市旁）,7.745038
2,10029618,近29~56天新开,其他,20250818,2025-08-18,30265585,傅鑫,中B上海V客销售组,中B上海V客连锁2组,ROM_3rd,...,0.0,0.0,哈灵现炒本帮面（天山西路店）,121.363920,31.218118,天山西路416号天山西路416号。,121.363791,31.218191,天山西路416号,14.726050
2,10029618,近29~56天新开,其他,20250818,2025-08-18,30265585,傅鑫,中B上海V客销售组,中B上海V客连锁2组,Behavior_AbnAppInstall,...,0.0,0.0,哈灵现炒本帮面（天山西路店）,121.363920,31.218118,天山西路416号天山西路416号。,121.363791,31.218191,天山西路416号,14.726050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18460,27224559,近28天新开,其他,20250923,2025-09-23,32284911,李群勇,武汉二区,武汉2区3组,Behavior_DevMode,...,0.0,0.0,豪记蛋炒饭摊,114.181506,30.577404,永丰街道仙山新城汉阳仙山新城18栋104,114.181506,30.577404,永丰街道仙山新城,0.000000
18461,27224642,近28天新开,其他,20250923,2025-09-23,32306991,刘建国,上海一区,上海1区1组,Behavior_AbnAppInstall,...,0.0,0.0,浏阳蒸菜(曹杨路店),121.421401,31.234239,长寿路街道浏阳蒸菜(曹杨路店)曹杨路302号,121.421426,31.234224,曹杨路450号绿地和创大厦B1楼A-14室,2.907014
18461,27224642,近28天新开,其他,20250923,2025-09-23,32306991,刘建国,上海一区,上海1区1组,Behavior_DeviceReset,...,0.0,0.0,浏阳蒸菜(曹杨路店),121.421401,31.234239,长寿路街道浏阳蒸菜(曹杨路店)曹杨路302号,121.421426,31.234224,曹杨路450号绿地和创大厦B1楼A-14室,2.907014
18462,27224675,近28天新开,其他,20250923,2025-09-23,32434331,周俊杰,厦门分区,厦门3组,APP_AutoClick,...,0.0,0.0,千里香馄饨,118.054285,24.560079,杏林街道高浦北路67-101号杏林街道高浦北路67-101号,118.054285,24.560079,杏林街道高浦北路67-101号,0.000000


In [34]:
# 不同二级标签下客户数
second_tag_cust_cnt = expand_df.groupby(['level2Tags']).agg({"customer_id":'nunique'}).sort_values(by='customer_id',ascending=False)
second_tag_cust_cnt[:10]

,customer_id
level2Tags,
Behavior_AbnAppInstall,3712
Execution_MultiSys,2798
Behavior_DeviceReset,2349
Behavior_AbnBattery,2265
Execution_MultiApp,941
Behavior_DevMode,841
APP_AutoClick,795
Execution_Root,759
Parameter_Mismatch,534


In [35]:
second_tag_cust_cnt
(second_tag_cust_cnt['customer_id'] >= 10).index

,customer_id
level2Tags,
Behavior_AbnAppInstall,3712
Execution_MultiSys,2798
Behavior_DeviceReset,2349
Behavior_AbnBattery,2265
Execution_MultiApp,941
Behavior_DevMode,841
APP_AutoClick,795
Execution_Root,759
Parameter_Mismatch,534


Index(['Behavior_AbnAppInstall', 'Execution_MultiSys', 'Behavior_DeviceReset',
       'Behavior_AbnBattery', 'Execution_MultiApp', 'Behavior_DevMode',
       'APP_AutoClick', 'Execution_Root', 'Parameter_Mismatch',
       'Parameter_Falsify', 'Execution_MockLocation', 'Behavior_Algorithm',
       'Parameter_LID1toN', 'Parameter_IdNto1', 'ROM_Modified',
       'Execution_Inject', 'ROM_3rd', 'Device_Emulator', 'APP_Hook',
       'APP_Location', 'APP_Root', 'Execution_TamperLocation',
       'Device_VMCloudPhone', 'Behavior_SuspectedAttack', 'Parameter_HID1toN',
       'APP_SysModify', 'Behavior_EnvDebug', 'Execution_MalFile',
       'Execution_HiddenRoot', 'Execution_Hook', 'Network_MITM'],
      dtype='object', name='level2Tags')

In [36]:
# Behavior_AbnAppInstall ✅
# Execution_MultiSys ✅
# Behavior_DeviceReset

In [37]:
# 以Behavior_AbnAppInstall为例
# second_expand_df = expand_df[expand_df['level2Tags'] == "Behavior_DeviceReset"].drop_duplicates()
second_expand_df = expand_df[expand_df['level2Tags'].isin((second_tag_cust_cnt['customer_id'] >= 10).index)].drop_duplicates()
second_expand_bd = second_expand_df.groupby(['new_begin_period', 'first_bd_name',"area_name","org_name"]).agg({'customer_id': 'nunique'}).reset_index()
second_expand_bd.columns = ['new_begin_period', 'first_bd_name',"area_name","org_name",'second_expand_bd_cnt']
second_expand_df = pd.merge(second_expand_df,second_expand_bd,on=['new_begin_period', 'first_bd_name',"area_name","org_name"],how='left').sort_values('first_bd_name', ascending=False)

In [38]:
# 当前二级标签下BD，上阶段新开商户聚集超过X以上的BD
# second_expand_bd_list = second_expand_df.loc[(second_expand_df['new_begin_period'] == "近29~56天新开") & (second_expand_df['second_expand_bd_cnt'] >= 5),'first_bd_name'].unique()
# second_expand_df.loc[second_expand_df['first_bd_name'].isin(second_expand_bd_list),:].to_excel("安装设备异常BD名下聚集客户.xlsx",index=False)

In [39]:
# second_expand_df.head()
second_expand_df.loc[(second_expand_df['new_begin_period'] == "近28天新开") & (second_expand_df['is_achieve_new_period']=="走完新客期"),"customer_id"].nunique()

272

In [40]:
# 计算近28天新开客户和近29～56天前BD名下客户距离
# second_expand_df = second_expand_df.loc[second_expand_df['first_bd_name'].isin(second_expand_bd_list),:]
# second_expand_df_28d = second_expand_df[second_expand_df['new_begin_period'] == "近28天新开"] # 不限制走完新客期
second_expand_df_28d = second_expand_df[(second_expand_df['new_begin_period'] == "近28天新开") & (second_expand_df['is_achieve_new_period']=="走完新客期")] # 限制走完新客期
second_expand_df_28d_copy = second_expand_df.loc[(second_expand_df['new_begin_period'] == "近28天新开"),['customer_id','first_arranged_ord_date','first_arranged_ord_belong_bd_id',"delivery_address","longitude","latitude"]] # 限制走完新客期
second_expand_df_28d_copy = second_expand_df_28d_copy.rename(columns={"customer_id":"related_customer_id",'first_arranged_ord_date':'related_cust_first_ord_date','delivery_address':'related_address','longitude':'related_longitude','latitude':'related_latitude'})
second_expand_df_56d = second_expand_df.loc[second_expand_df['new_begin_period'] == "近29~56天新开",['customer_id','first_arranged_ord_date','first_arranged_ord_belong_bd_id',"delivery_address","longitude","latitude"]]
second_expand_df_56d = second_expand_df_56d.rename(columns={"customer_id":"related_customer_id",'first_arranged_ord_date':'related_cust_first_ord_date','delivery_address':'related_address','longitude':'related_longitude','latitude':'related_latitude'})
second_cust_merge_new_new = pd.merge(second_expand_df_28d,second_expand_df_28d_copy,on=['first_arranged_ord_belong_bd_id'],how='left')
second_cust_merge_new_old = pd.merge(second_expand_df_28d,second_expand_df_56d,on=['first_arranged_ord_belong_bd_id'],how='left')

second_cust_merge = pd.concat([second_cust_merge_new_new,second_cust_merge_new_old])
second_cust_merge.shape
second_cust_merge = second_cust_merge[second_cust_merge['customer_id'] != second_cust_merge['related_customer_id']]
second_cust_merge.shape
second_cust_merge.drop_duplicates(inplace=True)
second_cust_merge.shape
second_cust_merge.head()

(14334, 34)

(13123, 34)

(4731, 34)

,customer_id,new_begin_period,is_achieve_new_period,first_arranged_ord_dt,first_arranged_ord_date,first_arranged_ord_belong_bd_id,first_bd_name,area_name,org_name,level2Tags,...,poi_longitude,poi_latitude,poi_address,poi_delivery_distance,second_expand_bd_cnt,related_customer_id,related_cust_first_ord_date,related_address,related_longitude,related_latitude
1,24288664,近28天新开,走完新客期,20250827,2025-08-27,32132429,龚春华,深圳一区,深圳1区4组,Parameter_Mismatch,...,113.881405,22.575652,西乡街道东村,870.376859,4.0,26942582.0,2025-08-29,西乡街道径贝饭堂门径贝饭堂门12栋6号,113.880079,22.570109
13,24288664,近28天新开,走完新客期,20250827,2025-08-27,32132429,龚春华,深圳一区,深圳1区4组,Parameter_Mismatch,...,113.881405,22.575652,西乡街道东村,870.376859,4.0,26936154.0,2025-08-28,西乡街道下三村小区下三村小区四排4号,113.859591,22.580968
14,24288664,近28天新开,走完新客期,20250827,2025-08-27,32132429,龚春华,深圳一区,深圳1区4组,Parameter_Mismatch,...,113.881405,22.575652,西乡街道东村,870.376859,4.0,26822390.0,2025-09-03,西乡街道下三村小区牛湾路6号旁,113.859986,22.581175
24,24288664,近28天新开,走完新客期,20250827,2025-08-27,32132429,龚春华,深圳一区,深圳1区4组,Behavior_DeviceReset,...,113.881405,22.575652,西乡街道东村,870.376859,4.0,26942582.0,2025-08-29,西乡街道径贝饭堂门径贝饭堂门12栋6号,113.880079,22.570109
36,24288664,近28天新开,走完新客期,20250827,2025-08-27,32132429,龚春华,深圳一区,深圳1区4组,Behavior_DeviceReset,...,113.881405,22.575652,西乡街道东村,870.376859,4.0,26936154.0,2025-08-28,西乡街道下三村小区下三村小区四排4号,113.859591,22.580968


In [41]:
import os 
os.getcwd()

'/Users/lizhongtao/myworkspace/SQL优化/设备风险画像监控/设备风险画像-BD维度聚合的新开客户'

In [42]:
# BD名下近两个月超过10个命中标签的新开客户
# 28天前新开客户关联客户超过3个；
# 距离50m以内；
# 相似度大于等于50%；

In [43]:
new_cust_related_cnt = second_cust_merge['customer_id'].value_counts().to_frame().reset_index()
new_cust_related_cnt.shape
new_cust_related_cnt_3 = new_cust_related_cnt.loc[new_cust_related_cnt['count'] >= 3,'customer_id'].values.tolist()
len(new_cust_related_cnt_3)

(272, 2)

187

In [44]:
from distance_calculate import GeoDistanceCalculator
from text_similar import * 
second_cust_merge['新老客收货距离'] = second_cust_merge.apply(lambda x: GeoDistanceCalculator.calculate_distance(x['latitude'], x['longitude'], x['related_latitude'], x['related_longitude']), axis=1)
second_cust_merge['delivery_address'] = second_cust_merge['delivery_address'].astype(str)
second_cust_merge['related_address'] = second_cust_merge['related_address'].astype(str)
second_cust_merge['新老客收货地址相似度'] = second_cust_merge.apply(lambda x: calculate_similarity_score(x['delivery_address'], x['related_address']),axis=1)

In [45]:
import numpy as np
pd.cut(second_cust_merge['新老客收货距离'],bins=[-1,100,200,300,400,500,600,700,800,900,1000,1500,2000,3000,5000,10000,np.inf]).value_counts().sort_index()

新老客收货距离
(-1.0, 100.0]         647
(100.0, 200.0]        289
(200.0, 300.0]        103
(300.0, 400.0]         73
(400.0, 500.0]         98
(500.0, 600.0]         84
(600.0, 700.0]        141
(700.0, 800.0]        100
(800.0, 900.0]        100
(900.0, 1000.0]        49
(1000.0, 1500.0]      212
(1500.0, 2000.0]      257
(2000.0, 3000.0]      369
(3000.0, 5000.0]      563
(5000.0, 10000.0]     476
(10000.0, inf]       1069
Name: count, dtype: int64

In [46]:
second_cust_merge.columns

Index(['customer_id', 'new_begin_period', 'is_achieve_new_period',
       'first_arranged_ord_dt', 'first_arranged_ord_date',
       'first_arranged_ord_belong_bd_id', 'first_bd_name', 'area_name',
       'org_name', 'level2Tags', 'shibie_uuid_cnt', 'new_cust_begin_dt',
       'is_yellow_line_cust', 'is_immediate_loss_process_index_cust',
       'is_immediate_loss_newa_cust', 'is_strategy_triggered',
       'is_related_cust_newa_or_process_index',
       'is_immediate_loss_newa_or_process_1arranged',
       'is_non_visit_app_after_1arranged', 'is_loss_newa_cust_cat2_cluster',
       'poi_name', 'longitude', 'latitude', 'delivery_address',
       'poi_longitude', 'poi_latitude', 'poi_address', 'poi_delivery_distance',
       'second_expand_bd_cnt', 'related_customer_id',
       'related_cust_first_ord_date', 'related_address', 'related_longitude',
       'related_latitude', '新老客收货距离', '新老客收货地址相似度'],
      dtype='object')

In [47]:
# BD维度下计算客户之间距离，筛选出距离小于X米的
tmp_merge = second_cust_merge.loc[second_cust_merge['新老客收货距离'] <= 200,['customer_id',"poi_name",'new_begin_period',"first_arranged_ord_dt","first_bd_name","org_name","is_yellow_line_cust","is_strategy_triggered","delivery_address","poi_delivery_distance",
                                                           "related_customer_id","related_cust_first_ord_date","related_address","新老客收货距离","新老客收货地址相似度"]].sort_values("新老客收货距离")
tmp_merge['customer_id'].nunique()
tmp_merge.loc[tmp_merge['is_strategy_triggered'] == 1,'customer_id'].nunique() 

70

16

In [48]:
# tmp_merge.loc[tmp_merge['is_strategy_triggered'] != 1,:].head()
tmp_merge['数值相似度'] = tmp_merge['新老客收货地址相似度'].apply(lambda x : 1 if x in ["包含","相同"] else x)
same_text_cnt = tmp_merge.loc[(tmp_merge['is_strategy_triggered'] != 1) & (tmp_merge['新老客收货地址相似度'] == "相同"),'customer_id'].nunique()
contain_text_cnt = tmp_merge.loc[(tmp_merge['is_strategy_triggered'] != 1) & (tmp_merge['新老客收货地址相似度'] == "包含"),'customer_id'].nunique()
similar_90_cnt = tmp_merge.loc[(tmp_merge['is_strategy_triggered'] != 1) & (tmp_merge['数值相似度']  >= 0.9),'customer_id'].nunique()
print(f"收货地址相同：{same_text_cnt}，收货地址包含：{contain_text_cnt}，收货地址相似度90%以上：{similar_90_cnt}")
same_contain_similar_cnt = tmp_merge.loc[(tmp_merge['is_strategy_triggered'] != 1) & ((tmp_merge['数值相似度']  >= 0.9) | (tmp_merge['新老客收货距离'] <= 100)),'customer_id'].nunique()
print(f"（剔除红线已识别）收货地址相同或包含或相似度90%以上且收货距离小于100：{same_contain_similar_cnt}")
tmp_merge.loc[(tmp_merge['is_strategy_triggered'] != 1) & ((tmp_merge['数值相似度']  >= 0.9) | (tmp_merge['新老客收货距离'] <= 100)) ,:]

收货地址相同：8，收货地址包含：5，收货地址相似度90%以上：22
（剔除红线已识别）收货地址相同或包含或相似度90%以上且收货距离小于100：47


,customer_id,poi_name,new_begin_period,first_arranged_ord_dt,first_bd_name,org_name,is_yellow_line_cust,is_strategy_triggered,delivery_address,poi_delivery_distance,related_customer_id,related_cust_first_ord_date,related_address,新老客收货距离,新老客收货地址相似度,数值相似度
7131,26877912,一品串坊老味炸串,近28天新开,20250827,全利,天津飞龙1组,1.0,0.0,兴南街道新泉大厦家佳星厨一楼,5.186113,26806666.0,2025-08-19,兴南街道新泉大厦家佳星厨一楼,0.000000,相同,1.000000
4937,23865042,刚好快餐店,近28天新开,20250827,彭彬,北京2区4组,1.0,0.0,星厨美食汇,16426.495217,26850823.0,2025-08-25,清源街道客隆家家综合超市入口旁星厨美食汇原健康菜,0.000000,包含,1.000000
4955,23865042,刚好快餐店,近28天新开,20250827,彭彬,北京2区4组,1.0,0.0,星厨美食汇,16426.495217,26850823.0,2025-08-25,清源街道客隆家家综合超市入口旁星厨美食汇原健康菜,0.000000,包含,1.000000
1250,26795285,阿甘锅盔摊（飞洲国际店）,近28天新开,20250827,陈秋静,上海2区4组,1.0,0.0,徐家汇街道美城天地(嘉汇国际广场店)零陵路899号,744.323669,25699518.0,2025-08-18,徐家汇街道美城天地(嘉汇国际广场店)肇嘉浜路825号,0.000000,0.848485,0.848485
4931,23865042,刚好快餐店,近28天新开,20250827,彭彬,北京2区4组,1.0,0.0,星厨美食汇,16426.495217,26850798.0,2025-08-26,星厨美食汇星厨美食汇150米健康菜,0.000000,包含,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4225,26881322,临沂炒鸡酸菜鱼摊,近28天新开,20250827,朱文康,济南4组,0.0,0.0,唐冶街道鲁商天街1111,0.000000,26872601.0,2025-08-26,唐冶街道鲁商天街1111,188.825226,相同,1.000000
4175,26881322,临沂炒鸡酸菜鱼摊,近28天新开,20250827,朱文康,济南4组,0.0,0.0,唐冶街道鲁商天街1111,0.000000,26872601.0,2025-08-26,唐冶街道鲁商天街1111,188.825226,相同,1.000000
3654,26881322,临沂炒鸡酸菜鱼摊,近28天新开,20250827,朱文康,济南4组,0.0,0.0,唐冶街道鲁商天街1111,0.000000,27054133.0,2025-09-09,唐冶街道鲁商天街11111,194.971485,包含,1.000000
3684,26881322,临沂炒鸡酸菜鱼摊,近28天新开,20250827,朱文康,济南4组,0.0,0.0,唐冶街道鲁商天街1111,0.000000,27054133.0,2025-09-09,唐冶街道鲁商天街11111,194.971485,包含,1.000000


In [49]:
new_cust_related_cnt = tmp_merge['customer_id'].value_counts().to_frame().reset_index()
new_cust_related_cnt.shape
new_cust_related_cnt_3 = new_cust_related_cnt.loc[new_cust_related_cnt['count'] >= 3,'customer_id'].values.tolist()
len(new_cust_related_cnt_3)

(70, 2)

54

In [50]:
new_cust_related_cnt = tmp_merge.loc[((tmp_merge['数值相似度']  >= 0.5) | (tmp_merge['新老客收货距离'] <= 50)) ,"customer_id"].value_counts().to_frame().reset_index()
new_cust_related_cnt.shape
new_cust_related_cnt_3 = new_cust_related_cnt.loc[new_cust_related_cnt['count'] >= 3,'customer_id'].unique()
len(new_cust_related_cnt_3)

(70, 2)

54

In [57]:
new_cust_related_cnt

,customer_id,count
0,26881889,115
1,26878471,105
2,26881322,54
3,23649279,54
4,26882004,52
...,...,...
65,26805310,1
66,26858353,1
67,26779063,1
68,26881531,1


In [60]:
# tmp_merge.loc[(tmp_merge['is_strategy_triggered'] != 1) & (tmp_merge['数值相似度']  >= 0.9),:].to_excel("tmp.xlsx",index=False)
tmp_merge.loc[(tmp_merge['is_strategy_triggered'] != 1) & ((tmp_merge['数值相似度']  >= 0.9) | (tmp_merge['新老客收货距离'] <= 200)) ,:].drop_duplicates().to_excel("tmp200.xlsx",index=False)
tmp_merge.loc[(tmp_merge['customer_id'].isin(new_cust_related_cnt_3)) & ((tmp_merge['数值相似度']  >= 0.5) | (tmp_merge['新老客收货距离'] <= 50)) ,:].drop_duplicates().to_excel("tmp0926_2.xlsx",index=False)

In [64]:
df.loc[df['customer_id'] == 24189734,'level2Tags'].values

array([list(['Device_VMCloudPhone', 'Execution_Root', 'Behavior_DeviceReset', 'ROM_Modified'])],
      dtype=object)

In [ ]:
# 真实
26854136
26854497 # Behavior_DeviceReset
26854765 # Behavior_AbnAppInstall, Behavior_DeviceReset
26864734 # Behavior_AbnAppInstall, Parameter_Mismatch
26876431 # Behavior_AbnAppInstall, Behavior_DeviceReset
26878132 # Behavior_AbnAppInstall, Execution_MultiApp
26880105 # Behavior_AbnAppInstall

# 虚假
19431668 # Behavior_AbnAppInstall, Behavior_DeviceReset
21806994 # Behavior_DeviceReset, Behavior_AbnAppInstall
23649279 # list(['Behavior_Algorithm', 'Execution_MultiSys', 'Execution_MultiApp', 'Behavior_AbnAppInstall', 'Behavior_SuspectedAttack', 'Parameter_Mismatch'])
23865042 # ['Behavior_DeviceReset', 'Execution_MultiSys']
24189734 # ['Device_VMCloudPhone', 'Execution_Root', 'Behavior_DeviceReset', 'ROM_Modified']
24406429
25133202
25750316
26627199
26670073
26710918
26770769
26795285
26828383
26859838
26861050
26870694
26872063
26873290
26877912
26878471
26880587
26881322
26881748
26882004
26882138
26882501